In [4]:
!pip install langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 24.6 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.44
    Uninstalling langsmith-0.3.44:
      Successfully uninstalled langsmith-0.3.44
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63


In [10]:
!pip install langgraph


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 20.0 MB/s eta 0:00:00


In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import StateGraph, START
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage
import os

In [64]:
os.environ["OPENAI_API_KEY"] = ""


In [65]:
def add(a: int, b: int) -> int:
    """Adds two integers."""
    return a + b

def multiply(a: int, b: int) -> int:
    """Multiplies two integers."""
    return a * b

def divide(a: int, b: int) -> float:
    """Divides two numbers."""
    return a / b

In [66]:
llm = ChatOpenAI(model="gpt-4o")
tools = [add,multiply,divide]
llm_with_tools = llm.bind_tools(tools)

In [67]:
sys_msg  = SystemMessage(
    content="You are a helpful budget assistant. Use tools to calculate expenses. Do not answer directly without tools."
)

In [68]:
def assistant(state: MessagesState):
  return {"messages":[llm_with_tools.invoke([sys_msg ]+state["messages"])]}


In [69]:
tool_node = ToolNode(tools)

In [70]:
builder = StateGraph(MessagesState)

builder.add_node("assistant", assistant)
builder.add_node("tools", tool_node)

builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")

#add memory
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)


In [71]:
config = {"configurable": {"thread_id": "budget_raghad"}}


In [ ]:
msg = [HumanMessage(content="Add 400 (rent) and 120 (groceries).")]
result = graph.invoke({"messages": msg}, config)

for m in result["messages"]:
    m.pretty_print()


msg = [HumanMessage(content="Now multiply the total by 1.15 for tax.")]
result = graph.invoke({"messages": msg}, config)

for m in result["messages"]:
    m.pretty_print()


msg = [HumanMessage(content="Now divide that by 30 days.")]
result = graph.invoke({"messages": msg}, config)

for m in result["messages"]:
    m.pretty_print()